In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import ipywidgets as widgets

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
#city_data_df.head()

In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
#Get the latitidue and longitude of each city
locations = city_data_df[["Lat","Lng"]]
#set any negative temperatures to 0F (shouldn't it be better to use Kelvin which is an absolute temp?)
city_data_df["Max Temp"] = city_data_df["Max Temp"].clip(lower=0)
#Get the max temperatures of each city
max_temp = city_data_df["Max Temp"]
# assign the gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# assign the heatmap
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating = False, 
                                    max_intensity=300,point_radius=4)
# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
temp_range_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp),:]
temp_range_df = temp_range_df.dropna()
temp_range_df.head(5)

In [ ]:
temp_range_df.info()

In [ ]:
hotel_df = temp_range_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [ ]:
def find_hotel(row) :
    lat = row["Lat"]
    lng = row["Lng"]
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key,
        "location": f"{lat},{lng}"}
    r = requests.get(base_url,params).json()
    try :
        row["Hotel Name"] = r["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping")
    return row

hotel_df = hotel_df.apply(find_hotel,axis=1)
hotel_df.head()

In [ ]:
indicies = hotel_df[hotel_df["Hotel Name"] ==""].index
hotel_df = hotel_df.drop(index=indicies)
hotel_df.info()

In [ ]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [ ]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig